# Packages

In [ ]:
!pip install transformers torch accelerate requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 824.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

# Screen OFAC

In [ ]:
import requests
import json

def screen_entities_ofac(cases):
    API_KEY = ""

    payload = {
        "apiKey": API_KEY,
        "minScore": 95,
        "source": ["all"],
        "types": ["person", "organization"],
        "cases": cases
    }

    headers = {"Content-Type": "application/json"}
    url = "https://api.ofac-api.com/v4/screen"

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        result = response.json()
        return result
    else:
        print("Error:", response.status_code, response.text)
        return None

# OpenSanction Screen

In [ ]:
import requests
import json

# OpenSanctions API Key
API_KEY = ""

# API Endpoint for sanctions screening
API_URL = "https://api.opensanctions.org/match/sanctions"

def screen_entities_openSanctionsAPI(cases):
    headers = {
        "Accept": "application/json",
        "Authorization": f"ApiKey {API_KEY}"
    }

    # Construct JSON request payload
    payload = {
        "queries": {
            f"q{i+1}": {
                "schema": "Person" if case["type"] == "person" else "Company",
                "properties": {"name": [case["name"]]}
            }
            for i, case in enumerate(cases)
        }
    }

    # Send POST request to OpenSanctions API
    response = requests.post(API_URL, headers=headers, json=payload)

    # Handle response
    if response.status_code == 200:
        data = response.json().get("responses", {})
        results = {case["name"]: data.get(f"q{i+1}", {}).get("results", []) for i, case in enumerate(cases)}
        return results
    else:
        print(f"Error fetching data: {response.status_code} - {response.text}")
        return None


# LLM Model for Summary

In [ ]:
import json
import google.generativeai as genai

def risk_analysis_llm(ofac_response, open_sanctions_response, model_name="gemini-1.5-pro", max_tokens=1000, temperature=0.7):
    # Combine both responses into a single JSON structure
    combined_data = {
        "OFAC Screening Results": ofac_response,
        "OpenSanctions Screening Results": open_sanctions_response
    }

    # Convert JSON to formatted string
    json_output_str = json.dumps(combined_data, indent=2)

    prompt = f"""You are a risk analysis expert specializing in identifying potential financial, legal, and security risks based on sanctions data that I will be providing you. You will carefully analyze the following raw JSON response, which contains details about various entities from multiple sources (OFAC and OpenSanctions). Your task is to determine whether the entities involved are risky and, if so, provide a well-reasoned justification with supporting evidences. You be like an Explainable AI.
A possbile response format can be something like:
Sanction Analysis:

Entity Name
- Risk Score: (0-1)?
- Justification and Evidence: (should include List of sanctioning bodies and any other details available, Reason for sanctions such as Terrorism, money laundering, fraud, etc., and any Additional insights like alias details, past violations, associated organizations, if any)

... (if more entities involved, use the same format)

Overall Risk Score (Sanction based) for the transaction:
- Final Risk Level: (in 0-1)
- Confidence Level: (in 0-1;  how much confidence do you have on the Final Risk Level)
- Justification: Justify everything, be as much detailed as possible, consideration of all individual entity risk levels, explanation of why this transaction should be flagged or cleared, Any patterns or red flags in the provided data, or anything. Think out of the box and think like an expert

Here is the JSON output from the sanctions screening:
{json_output_str}
"""

    # Configure the Gemini API
    genai.configure(api_key="")

    # Select the Gemini model
    model = genai.GenerativeModel(model_name)

    # Generate the response
    response = model.generate_content(prompt, generation_config={"temperature": temperature, "max_output_tokens": max_tokens})

    return response.text if response else "Error: No response from the model"


# Main

In [68]:
cases = [
    {"name": "Acme Corporation", "type": "organization"},
    {"name": "Oceanic Holdings LLC", "type": "organization"},
]

screening_result_from_ofac = screen_entities_ofac(cases)
screening_result_from_openSanctionsAPI = screen_entities_openSanctionsAPI(cases)

response = risk_analysis_llm(screening_result_from_ofac, screening_result_from_openSanctionsAPI)
print(response)


Sanction Analysis:

**Acme Corporation**
- Risk Score: 0.73
- Justification and Evidence:
    - Sanctioning Body: OpenSanctions (ch_seco_sanctions dataset)
    - Reason for Sanction: Assa Corporation, which has a high similarity with Acme Corporation, is listed as a front company for Bank Melli, used to channel money from the United States to Iran.  This implies potential involvement in sanctions evasion and money laundering activities.
    - Additional Insights: The address listed for Assa Corporation is in New York, USA.  The listed Tax ID should be further investigated to confirm a link between Acme Corporation and Assa Corporation.

**Oceanic Holdings LLC**
- Risk Score: 0.70
- Justification and Evidence:
    - Sanctioning Body: OpenSanctions, referencing OFAC SDN (us_ofac_sdn dataset), PermID, US Trade CSL, and US SAM Exclusions.
    - Reason for Sanction: Cochan Holdings LLC, which has a high similarity with Oceanic Holdings LLC, is listed as an active sanctioned entity. The pres